# DenseNet: A Deep Learning Architecture

DenseNet, or Dense Convolutional Network, is a type of convolutional neural network (CNN) designed to improve the efficiency and performance of deep learning models. The **key idea** is the **dense connectivity pattern**, where **each layer is connected to every other layer in a feed-forward manner**.

##Key Features of DenseNet

1. Dense Connectivity:
    * Each layer gets inputs from all preceding layers and passes its output to all subsequent layers.
    * This ensures maximum information flow and helps mitigate the vanishing gradient problem.

2. Feature Reuse:
    * Instead of recalculating features, DenseNet reuses them, making the model more parameter-efficient.

3. Compact Model:
    * DenseNet requires fewer parameters and reduces the risk of overfitting.

4. Transition Layers:
    * To control the complexity of the network, transition layers are used between dense blocks, performing down-sampling with pooling layers.

4. Growth Rate (k):
    * The number of feature maps added by each layer is fixed and is known as the growth rate.

##Structure of DenseNet

  1. Dense Block: A sequence of densely connected layers.
  
  2. Transition Layer: A layer between dense blocks that reduces feature map dimensions.

Here’s how to implement DenseNet using PyTorch with a simple dataset like CIFAR-10.

In [1]:
# Step 1: Import Libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [2]:
# Step 2: Load CIFAR-10 Dataset

# Transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 Dataset
batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


100%|██████████| 170M/170M [00:02<00:00, 77.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Step 3: Define DenseNet Architecture

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(self._make_layer(in_channels + i * growth_rate, growth_rate))

    def _make_layer(self, in_channels, growth_rate):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        )

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            new_feature = layer(torch.cat(features, dim=1))
            features.append(new_feature)
        return torch.cat(features, dim=1)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.transition = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.transition(x)

class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, num_layers=[6, 12, 24, 16], num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        # Initial Convolution
        num_channels = 2 * growth_rate
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size=3, stride=1, padding=1, bias=False)

        # Dense Blocks and Transition Layers
        self.dense_blocks = nn.ModuleList()
        self.transition_layers = nn.ModuleList()

        for i, num_layer in enumerate(num_layers):
            self.dense_blocks.append(DenseBlock(num_channels, growth_rate, num_layer))
            num_channels += num_layer * growth_rate
            if i != len(num_layers) - 1:
                self.transition_layers.append(TransitionLayer(num_channels, num_channels // 2))
                num_channels //= 2

        # Final BatchNorm and Fully Connected Layer
        self.bn = nn.BatchNorm2d(num_channels)
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        for i, block in enumerate(self.dense_blocks):
            x = block(x)
            if i < len(self.transition_layers):
                x = self.transition_layers[i](x)
        x = self.bn(x)
        x = torch.relu(x)
        x = torch.mean(x, dim=[2, 3])  # Global Average Pooling
        x = self.fc(x)
        return x


In [ ]:
# Step 4: Train and Test the Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

import matplotlib.pyplot as plt

# Variables to store loss and accuracy for each epoch
train_losses = []
test_accuracies = []

for epoch in range(10):  # Number of epochs
    # Training phase
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    train_losses.append(avg_train_loss)

    # Testing phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    test_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}, Loss: {avg_train_loss:.4f}, Accuracy: {accuracy:.2f}%")


## Summary

 * DenseNet's key advantage is feature reuse and efficient use of parameters.
 * The above code implements DenseNet with CIFAR-10 as an example dataset.
 * The network comprises dense blocks and transition layers to balance depth and parameter efficiency.

In [ ]:
# Plot Training Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss", color='blue', marker='o')
plt.title("Training Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

# Plot Testing Accuracy
plt.figure(figsize=(10, 5))
plt.plot(test_accuracies, label="Testing Accuracy", color='green', marker='o')
plt.title("Testing Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid()
plt.show()
